To test if a general purpose sentiment analysis model can be used to predict the sentiment of political tweets.

In [ ]:
# Define Model
import torch
import torch.nn as nn
from tqdm import tqdm
from transformers import RobertaModel
from transformers import RobertaTokenizer
from roberta_classifer import RobertaClassifier
    
# Instantiate the model
model = RobertaClassifier()

# Load pre-trained weights
#model.load_state_dict(torch.load('final_roberta_classifier.pt'))

#model.to('cuda')
# Set the model to evaluation mode
model.eval()

In [ ]:
import torch.nn.functional as F

def roberta_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to('cuda') for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate confidence
    probs = F.softmax(all_logits, dim=1).cpu()

    return probs


In [ ]:
batch_size = 32
test_data = ["I want more gun control", "I'm against gun control"]
test_inputs, test_masks = model.preprocess_for_roberta(test_data)

# Create dataloader
test_dataloader = model.create_dataloader(test_inputs, test_masks, batch_size)

# Compute predicted probabilities on the test set
probs = torch.argmax(roberta_predict(model, test_dataloader), dim=1)



In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
labels_map = {0: "Negative", 1: "Positive"}

def create_prediction_dictionary(sentences, labels):
    prediction_dict = {}
    for i in range(len(sentences)):
        prediction_dict[sentences[i]] = labels_map[labels[i].item()]
    return prediction_dict




In [ ]:
sentiment_dict = create_prediction_dictionary(test_data, probs)
sentiment_dict